In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# upstage 불러오기
import os

analyzer_api = os.environ.get("UPSTAGE_API_KEY")
print(analyzer_api)

up_5YgukBM6H71vAgXEMOjKccsfJvxFr


# 0) 그래프 State 정의

In [3]:
from typing import TypedDict, Annotated, List, Dict
import operator
from langchain_core.documents import Document


class GraphState(TypedDict):
    filepath: Annotated[str, "filepath"]  # 원본 파일 경로
    
    analyzed_files : Annotated[List, "analyzed_files"]

    metadata: Annotated[List[Dict], operator.add]  # parsing metadata (api, model, usage)
    
    elements_from_parser: Annotated[List[Dict], "elements_from_parser"]  


In [4]:
state = GraphState(filepath=r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트")
state

{'filepath': 'C:\\Users\\user\\OneDrive\\바탕 화면\\BOAZ\\2025_분석_ADV session\\챗봇 프로젝트'}

# 2) Upstage Document parser 이용해 layout 분석

In [1]:
import os

folder_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\GUIDELINES"

# 폴더 내 파일 개수 확인
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(f"파일 개수: {len(file_list)}")

파일 개수: 95


In [7]:
import requests
import json
import os

DEFAULT_CONFIG = {
    "ocr": False,
    "coordinates": True,
    "output_formats": ["html", "text", "markdown"],
    "model": "document-parse",
    "base64_encoding": ["figure", "chart", "table"],
}

def analyze_layout(state):
    """
    GUIDELINES 폴더 내 논문 PDF를 Upstage API로 전송하여 레이아웃 분석을 수행하고,
    결과 JSON 파일들을 저장하는 함수. 기존 분석 파일이 있으면 API 요청을 건너뛴다.

    Returns:
    - dict: 성공적으로 분석된 파일들의 결과 JSON 파일 경로 목록.
    """
    docs_folder = os.path.join(state['filepath'], "GUIDELINES")  # GUIDELINES 폴더 사용
    output_folder = os.path.join(state['filepath'], "analysis_results")
    os.makedirs(output_folder, exist_ok=True)
    
    api_url = "https://api.upstage.ai/v1/document-ai/document-parse"
    api_key = os.environ.get("UPSTAGE_API_KEY")

    # GUIDELINES 폴더 내 PDF 파일 가져오기 (이름순 정렬 후 상위 10개)
    pdf_files = sorted([f for f in os.listdir(docs_folder) if f.endswith(".pdf")])
    output_paths = []
    metadata_list = []
    
    for pdf_file in pdf_files:
        file_path = os.path.join(docs_folder, pdf_file)
        output_json_path = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}.json")

        # JSON 파일이 이미 존재하면 API 요청을 건너뛰고 기존 데이터를 사용
        if os.path.exists(output_json_path):
            print(f"🟢 기존 분석 파일 사용: {output_json_path}")
            with open(output_json_path, "r", encoding="utf-8") as json_file:
                result = json.load(json_file)
            meta = {
                "id": pdf_file,
                "model": result.get("model"),
                "usage": result.get("usage")
            }
            metadata_list.append(meta)
            output_paths.append(output_json_path)
            continue

        # API 요청을 통한 새로운 분석 수행
        print(f"📤 파일 업로드 중: {pdf_file}")
        with open(file_path, "rb") as pdf:
            response = requests.post(
                api_url,
                headers={"Authorization": f"Bearer {api_key}"},
                json=DEFAULT_CONFIG,  # ✅ 여기 수정
                files={"document": pdf}
            )
        
        # 응답 확인 및 JSON 파일 저장
        if response.status_code == 200:
            result = response.json()
            with open(output_json_path, "w", encoding="utf-8") as json_file:
                json.dump(result, json_file, ensure_ascii=False, indent=4)
            print(f"✅ 분석 결과 저장 완료: {output_json_path}")
            output_paths.append(output_json_path)
            meta = {
                "id": pdf_file,
                "model": result.get("model"),
                "usage": result.get("usage")
            }
            metadata_list.append(meta)
        else:
            print(f"❌ 오류 발생 ({response.status_code}): {response.text}")
    
    print("🎉 모든 파일 분석 완료!")
    return {"metadata": metadata_list, "analyzed_files": output_paths}


In [8]:
# 실행
state_out = analyze_layout(state)
state.update(state_out)
state

🟢 기존 분석 파일 사용: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results\1-s2.0-S0952818016300204-main.json
🟢 기존 분석 파일 사용: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results\1-s2.0-S0952818022000101-main.json
🟢 기존 분석 파일 사용: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results\1-s2.0-S0952818022003701-main.json
🟢 기존 분석 파일 사용: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results\1-s2.0-S0952818024000333-main.json
🟢 기존 분석 파일 사용: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results\1-s2.0-S0952818024001375-main.json
🟢 기존 분석 파일 사용: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results\1-s2.0-S2352556824000626-main.json
🟢 기존 분석 파일 사용: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results\2022 American Society of Anesthesiologists Practice Guidelines for Management of the Difficult Airway.json
🟢 기존 분석 파일 사용: C:\Users\u

{'filepath': 'C:\\Users\\user\\OneDrive\\바탕 화면\\BOAZ\\2025_분석_ADV session\\챗봇 프로젝트',
 'metadata': [{'id': '1-s2.0-S0952818016300204-main.pdf',
   'model': 'document-parse-250116',
   'usage': {'pages': 12}},
  {'id': '1-s2.0-S0952818022000101-main.pdf',
   'model': 'document-parse-250116',
   'usage': {'pages': 9}},
  {'id': '1-s2.0-S0952818022003701-main.pdf',
   'model': 'document-parse-250116',
   'usage': {'pages': 9}},
  {'id': '1-s2.0-S0952818024000333-main.pdf',
   'model': 'document-parse-250116',
   'usage': {'pages': 6}},
  {'id': '1-s2.0-S0952818024001375-main.pdf',
   'model': 'document-parse-250116',
   'usage': {'pages': 10}},
  {'id': '1-s2.0-S2352556824000626-main.pdf',
   'model': 'document-parse-250116',
   'usage': {'pages': 13}},
  {'id': '2022 American Society of Anesthesiologists Practice Guidelines for Management of the Difficult Airway.pdf',
   'model': 'document-parse-250116',
   'usage': {'pages': 51}},
  {'id': '635.full.pdf',
   'model': 'document-parse-2501

In [9]:
import os

# 분석 결과가 저장된 폴더 경로
analysis_folder = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results"

# ✅ .json 파일 목록 추출
analyzed_jsons = [f for f in os.listdir(analysis_folder) if f.endswith(".json")]

# ✅ 파일명 출력
print("✅ 분석 완료된 JSON 파일 개수:", len(analyzed_jsons))
for filename in analyzed_jsons:
    print("📄", filename)


✅ 분석 완료된 JSON 파일 개수: 36
📄 1-s2.0-S0952818016300204-main.json
📄 1-s2.0-S0952818022000101-main.json
📄 1-s2.0-S0952818022003701-main.json
📄 1-s2.0-S0952818024000333-main.json
📄 1-s2.0-S0952818024001375-main.json
📄 1-s2.0-S2352556824000626-main.json
📄 2022 American Society of Anesthesiologists Practice Guidelines for Management of the Difficult Airway.json
📄 635.full.json
📄 A review of pediatric fasting guidelines and strategies to help children manage preoperative fasting.json
📄 Airway management in neonates and infants European Society of Anaesthesiology and Intensive Care and British Journal of Anaesthesia joint guidelines.json
📄 Airway management in patients with suspected or confirmed cervical spine injury Guidelines from.json
📄 An international multidisciplinary consensus statement on fasting before procedural sedation in adults and children.json
📄 Anesthetic Implications of the New Guidelines for Button Battery Ingestion in Children.json
📄 APM-13-018.json
📄 APM-14-380.json
📄 Current

In [17]:
from dotenv import load_dotenv
load_dotenv(override=True)  # 기존에 이미 설정된 값도 덮어씀!

True

In [18]:
# upstage 불러오기
import os

analyzer_api = os.environ.get("UPSTAGE_API_KEY")
print(analyzer_api)

up_eJgBUIf3QwcpDj3hqTib5pwQ4lUGY


In [23]:
import os
import requests
import json

# 경로 설정
guideline_dir = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\GUIDELINES"
analysis_dir = os.path.join(guideline_dir, "..", "analysis_results")
os.makedirs(analysis_dir, exist_ok=True)

# API 설정
api_url = "https://api.upstage.ai/v1/document-ai/document-parse"
api_key = os.environ["UPSTAGE_API_KEY"]

DEFAULT_CONFIG = {
    "ocr": False,
    "coordinates": True,
    "output_formats": ["html", "text", "markdown"],
    "model": "document-parse",
    "base64_encoding": ["figure", "chart", "table"],
}

# 파일 목록 정리
all_pdfs = sorted([f for f in os.listdir(guideline_dir) if f.endswith(".pdf")])
existing_jsons = set([f.replace(".json", ".pdf") for f in os.listdir(analysis_dir) if f.endswith(".json")])
pdfs_to_analyze = [f for f in all_pdfs if f not in existing_jsons]

print(f"🟡 아직 분석되지 않은 PDF 파일 수: {len(pdfs_to_analyze)}")

# 🔐 안전한 파일명 생성 함수 추가
def safe_filename(pdf_file: str, max_length=150) -> str:
    import re

    # 예외적으로 너무 긴 파일 직접 짧게 지정
    if "Recommendations on RBC Transfusion Support" in pdf_file:
        return "Recommendations on RBC Transfusion Support in Children With Hematologic and Oncologic Diagnoses TAXI.json"
    
    # 일반 처리 (원래대로 파일명 그대로 → .json)
    name = pdf_file.replace(".pdf", ".json")
    name = re.sub(r'[\\/*?:"<>|]', "", name)  # 특수문자 제거
    return name # 혹시 모르니 150자 제한도 포함

# 업로드 시작
for pdf_file in pdfs_to_analyze:
    input_path = os.path.join(guideline_dir, pdf_file)

    # 🔄 파일명 안전하게 리네이밍
    safe_name = safe_filename(pdf_file)
    output_path = os.path.join(analysis_dir, safe_name)

    print(f"📤 업로드 시도: {pdf_file}")
    with open(input_path, "rb") as file:
        try:
            response = requests.post(
                api_url,
                headers={"Authorization": f"Bearer {api_key}"},
                json=DEFAULT_CONFIG,
                files={"document": file}
            )
        except Exception as e:
            print(f"❌ 요청 중 예외 발생: {e}")
            break

    if response.status_code == 200:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(response.json(), f, ensure_ascii=False, indent=4)
        print(f"✅ 분석 성공 및 저장 완료: {pdf_file}")
    elif response.status_code == 401:
        print("❌ ❗ API 키 사용 중단됨 (401 Unauthorized - 크레딧 부족?) → 자동 중단")
        break
    else:
        print(f"❌ 분석 실패: {pdf_file} ({response.status_code}) → {response.text}")


🟡 아직 분석되지 않은 PDF 파일 수: 16
📤 업로드 시도: Recommendations on RBC Transfusion Support in Children With Hematologic and Oncologic Diagnoses From the Pediatric Critical Care Transfusion and Anemia Expertise Initiative.pdf
✅ 분석 성공 및 저장 완료: Recommendations on RBC Transfusion Support in Children With Hematologic and Oncologic Diagnoses From the Pediatric Critical Care Transfusion and Anemia Expertise Initiative.pdf
📤 업로드 시도: The Society for Pediatric Anesthesia recommendations for the use of opioids in children during the perioperative period.pdf
✅ 분석 성공 및 저장 완료: The Society for Pediatric Anesthesia recommendations for the use of opioids in children during the perioperative period.pdf
📤 업로드 시도: apm-21021.pdf
✅ 분석 성공 및 저장 완료: apm-21021.pdf
📤 업로드 시도: apm-22215.pdf
✅ 분석 성공 및 저장 완료: apm-22215.pdf
📤 업로드 시도: apm-23123.pdf
✅ 분석 성공 및 저장 완료: apm-23123.pdf
📤 업로드 시도: kja-19169.pdf
✅ 분석 성공 및 저장 완료: kja-19169.pdf
📤 업로드 시도: kja-19174.pdf
✅ 분석 성공 및 저장 완료: kja-19174.pdf
📤 업로드 시도: kja-19305.pdf
✅ 분석 성공 및 저장 완료: kj

In [25]:
import os

# 분석 결과가 저장된 폴더 경로
analysis_folder = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\analysis_results"

# ✅ .json 파일 목록 추출
analyzed_jsons = [f for f in os.listdir(analysis_folder) if f.endswith(".json")]

# ✅ 파일명 출력
print("✅ 분석 완료된 JSON 파일 개수:", len(analyzed_jsons))
for filename in analyzed_jsons:
    print("📄", filename)

✅ 분석 완료된 JSON 파일 개수: 94
📄 1-s2.0-S0952818016300204-main.json
📄 1-s2.0-S0952818022000101-main.json
📄 1-s2.0-S0952818022003701-main.json
📄 1-s2.0-S0952818024000333-main.json
📄 1-s2.0-S0952818024001375-main.json
📄 1-s2.0-S2352556824000626-main.json
📄 2022 American Society of Anesthesiologists Practice Guidelines for Management of the Difficult Airway.json
📄 635.full.json
📄 A review of pediatric fasting guidelines and strategies to help children manage preoperative fasting.json
📄 Airway management in neonates and infants European Society of Anaesthesiology and Intensive Care and British Journal of Anaesthesia joint guidelines.json
📄 Airway management in patients with suspected or confirmed cervical spine injury Guidelines from.json
📄 An international multidisciplinary consensus statement on fasting before procedural sedation in adults and children.json
📄 Anesthetic Implications of the New Guidelines for Button Battery Ingestion in Children.json
📄 APM-13-018.json
📄 APM-14-380.json
📄 apm-210

# 3 각 파일들의 element 저장

In [27]:
import os
import json
import re

def extract_font_size(html):
    """
    HTML 문자열에서 'font-size' 값을 추출하는 함수
    """
    match = re.search(r"font-size:(\d+)px", html)
    return int(match.group(1)) if match else None

def load_elements(state):
    """
    JSON에서 elements 데이터를 읽어와서 모든 키-값을 유지한 채 저장하는 함수.
    'font-size'가 없으면 HTML에서 추출하여 추가함.
    
    Returns:
      dict: 업데이트된 "elements_from_parser" 포함 (원본 JSON 데이터 전체 유지)
    """
    elements_dict = {}

    for json_file in state.get("analyzed_files", []):
        file_name = os.path.basename(json_file)
        with open(json_file, "r", encoding="utf-8") as f:
            json_data = json.load(f)

        raw_elements = json_data.get("elements", [])  # elements 키에서 데이터 가져옴
        parsed_elements = []  # 최종 문서 요소 리스트

        for element in raw_elements:
            processed_element = element.copy()  # ✅ 모든 키-값 유지

            # ✅ "<br>" 태그 제거 후 줄바꿈 없이 문장 연결
            if "content" in processed_element:
                for key in ["text", "html", "markdown"]:
                    if key in processed_element["content"]:
                        processed_element["content"][key] = re.sub(r'<br>', ' ', processed_element["content"].get(key, "").strip())

            # ✅ 폰트 크기 추가 (style 필드가 없을 경우)
            if "style" not in processed_element:
                processed_element["style"] = {}

            # ✅ style에서 폰트 크기를 찾고, 없으면 HTML에서 추출
            font_size = processed_element["style"].get("font-size")
            if font_size is None and "html" in processed_element["content"]:
                font_size = extract_font_size(processed_element["content"]["html"])
                if font_size:
                    processed_element["style"]["font-size"] = font_size  # 🔹 style에 추가

            parsed_elements.append(processed_element)

        # ✅ 최종 저장된 요소 개수 확인
        print(f"✅ Number of parsed elements in {file_name}: {len(parsed_elements)}")

        elements_dict[file_name] = parsed_elements  # JSON 원본 구조 유지

    return {"elements_from_parser": elements_dict}


In [ ]:
state_out = load_elements(state)
state.update(state_out)
state['elements_from_parser']

✅ Number of parsed elements in 1-s2.0-S0952818016300204-main.json: 128
✅ Number of parsed elements in 1-s2.0-S0952818022000101-main.json: 145
✅ Number of parsed elements in 1-s2.0-S0952818022003701-main.json: 137
✅ Number of parsed elements in 1-s2.0-S0952818024000333-main.json: 116
✅ Number of parsed elements in 1-s2.0-S0952818024001375-main.json: 157
✅ Number of parsed elements in 1-s2.0-S2352556824000626-main.json: 280
✅ Number of parsed elements in 2022 American Society of Anesthesiologists Practice Guidelines for Management of the Difficult Airway.json: 665
✅ Number of parsed elements in 635.full.json: 222
✅ Number of parsed elements in A review of pediatric fasting guidelines and strategies to help children manage preoperative fasting.json: 407
✅ Number of parsed elements in APM-13-018.json: 68
✅ Number of parsed elements in APM-14-380.json: 150
✅ Number of parsed elements in Airway management in neonates and infants European Society of Anaesthesiology and Intensive Care and Brit

# 5) 표 csv로 저장

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def export_table_csv(state):
    """
    문서에서 추출한 테이블을 CSV 형식으로 저장하고,
    각 table 요소에 대해 "table_csv_path" 키에 CSV 파일 경로를 추가합니다.

    state["filepath"] 내부에 "table_csv" 폴더를 생성하여 CSV 파일들을 저장합니다.

    Returns:
      dict: 업데이트된 state 딕셔너리 (각 table 요소에 "table_csv_path" 추가)
    """

    docs_folder = state["filepath"]
    table_dir = os.path.join(docs_folder, "table_csv")
    os.makedirs(table_dir, exist_ok=True)

    for json_file in state["elements_from_parser"]:
        pdf_filename = os.path.splitext(json_file)[0]  # PDF 파일명 추출 (확장자 제거)

        for i, element in enumerate(state['elements_from_parser'][json_file]):
            # ✅ "category" 키를 사용하여 테이블 찾기
            element_category = element.get("category", "")
            page = element.get("page", "unknown")
            table_id = element.get("id", "unknown")  # JSON의 실제 "id" 값 가져오기
            html = element.get("content", {}).get("html", "")  # ✅ "content" 내부에서 "html" 가져오기

            # 🔹 페이지 번호 설정
            if page is None:
                page = "unknown"

            # 🔹 테이블 감지
            if element_category == "table":
                try:
                    if not html:
                        print(f"⚠️ 테이블 HTML이 없습니다 (ID: {table_id}, Page: {page})")[:10]
                        continue

                    # 🔹 테이블 HTML 출력 (디버깅용)
                    print(f"\n🔍 Processing table ID {table_id}, Page {page}")
                    print(f"📝 Extracted HTML (첫 500자):\n{html[:500]}")

                    soup = BeautifulSoup(html, "html.parser")

                    # 🔹 테이블 존재 여부 확인
                    tables = soup.find_all("table")
                    if not tables:
                        print(f"⚠️ No <table> found in HTML (ID: {table_id}, Page: {page})")
                        continue

                    print(f"✅ {len(tables)} 개의 테이블이 감지되었습니다.")

                    for table_index, table in enumerate(tables):
                        # ✅ BeautifulSoup을 사용하여 직접 테이블 변환
                        rows = []
                        for row in table.find_all("tr"):
                            cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                            rows.append(cols)

                        if not rows:
                            print(f"⚠️ 테이블 데이터가 없습니다 (ID: {table_id}, Page: {page})")
                            continue

                        df = pd.DataFrame(rows)

                        # ✅ 파일명 형식 개선 (JSON 실제 ID 사용)
                        csv_filename = f"{pdf_filename}_TABLE_Page_{page}_ID_{table_id}_{table_index}.csv"
                        csv_filepath = os.path.join(table_dir, csv_filename)

                        df.to_csv(csv_filepath, index=False, encoding="utf-8-sig")
                        print(f"✅ CSV 파일 생성 완료: {csv_filepath}")

                        if "table_csv_path" not in element:
                            element["table_csv_path"] = []
                        element["table_csv_path"].append(csv_filepath)

                except Exception as e:
                    print(f"❌ 테이블 파싱 중 오류 발생 (ID: {table_id}, Page: {page}): {str(e)}")
                    continue

    return {"elements_from_parser": state["elements_from_parser"]}


In [ ]:
state_out = export_table_csv(state)
state.update(state_out)
state['elements_from_parser']

In [ ]:
state.keys()

# 6) Documents 구성

- metadata : 파일명, type, page, png_file_path, table_csv_path, 원본 text
- content : markdown

In [ ]:
from langchain_core.documents import Document
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import re

# 📌 NLTK 불용어 로드
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

documents = []
export_categories = {'heading1', 'paragraph', 'list', 'table'}
keywords_patterns = ["keywords", "key words", "KEYWORDS"]

for json_file, elements in state['elements_from_parser'].items():
    pdf_filename = json_file.replace(".json", ".pdf")

    # 초기화
    paper_title = None
    author = None
    key_words = None
    abstract_text = None

    # 🔍 제목 후보 필터링
    title_candidates = []
    journal_name_blacklist = ["Journal", "Proceedings", "Transactions", "ScienceDirect"]

    for idx, element in enumerate(elements):
        category = element.get("category", "")
        text = element.get("content", {}).get("text", "").strip()
        style = element.get("style", {})
        font_size = style.get("font-size")
        page = element.get("page", 999)

        # 제목 후보 수집
        if category in ["paragraph", "heading1"] and font_size and page == 1:
            if any(word.lower() in text.lower() for word in journal_name_blacklist):
                continue
            title_candidates.append((font_size, len(text), idx, text))

    # ✅ 최적 제목 선택
    title_idx = None
    if title_candidates:
        title_candidates.sort(key=lambda x: (-x[0], -x[1]))
        _, _, title_idx, paper_title = title_candidates[0]

        # 제목 바로 뒤 텍스트 → 저자
        if title_idx + 1 < len(elements):
            author = elements[title_idx + 1].get("content", {}).get("text", "").strip()
        if (not author) and (title_idx + 2 < len(elements)):
            author = elements[title_idx + 2].get("content", {}).get("text", "").strip()


    # ✅ Keywords & Abstract 추출 (정확도 향상 최종 버전)
    for idx, element in enumerate(elements):
        text = element.get("content", {}).get("text", "").strip()
        text_lower = text.lower()

        # ------------------------------
        # ✅ Keywords 추출
        # ------------------------------
        if any(k in text_lower for k in keywords_patterns):
            # Case 1: 같은 줄에 키워드 포함 (e.g., "Keywords: A; B; C")
            if ":" in text:
                possible_keywords = text.split(":", 1)[-1].strip()
                if len(possible_keywords.split()) >= 2:  # 최소 2단어 이상이면 키워드로 판단
                    key_words = possible_keywords
                    continue  # 다음 루프로 skip

            # Case 2: 다음 블록부터 1~4개 확인
            keyword_lines = []
            for offset in range(1, 6):
                next_idx = idx + offset
                if next_idx >= len(elements):
                    break
                next_el = elements[next_idx]
                next_text = next_el.get("content", {}).get("text", "").strip()

                # 리스트 스타일 키워드 확인
                if not next_text:
                    break
                if len(next_text) > 300:
                    break  # 너무 긴 텍스트는 제외

                if any(sep in next_text for sep in [";", ",", "•"]) or len(next_text.split()) < 12:
                    keyword_lines.append(next_text)
                else:
                    break
            key_words = " ".join(keyword_lines).strip()

        # ------------------------------
        # ✅ Abstract 추출
        # ------------------------------
        if "abstract" in text_lower:
            next_idx = idx + 1
            if next_idx < len(elements):
                next_el = elements[next_idx]
                if next_el.get("category") == "paragraph":
                    abstract_text = next_el.get("content", {}).get("text", "").strip()

    # ------------------------------
    # 🔧 후처리: 키워드 클리닝
    # ------------------------------
    def clean_keywords(keywords):
        if not keywords:
            return None
        parts = re.split(r'[•;,\\n]', keywords)
        cleaned = [kw.strip() for kw in parts if kw.strip()]
        return ", ".join(cleaned)

    key_words = clean_keywords(key_words)

    # ------------------------------
    # ❗ TF-IDF 백업 키워드 추출
    # ------------------------------
    if not key_words and abstract_text:
        def extract_keywords(text, num_keywords=5):
            text = re.sub(r'\W+', ' ', text)
            words = [word for word in text.lower().split() if word not in stop_words]
            vectorizer = TfidfVectorizer(stop_words="english", max_features=50)
            tfidf_matrix = vectorizer.fit_transform([" ".join(words)])
            feature_array = vectorizer.get_feature_names_out()
            scores = tfidf_matrix.toarray().flatten()
            sorted_indices = scores.argsort()[::-1]
            top_keywords = [feature_array[i] for i in sorted_indices[:num_keywords]]
            return ", ".join(top_keywords)

        key_words = extract_keywords(abstract_text)

    # ✅ Document 생성
    for element in elements:
        category = element.get("category", "")
        if category not in export_categories:
            continue

        text_content = element.get("content", {}).get("text", "").strip()
        markdown_content = element.get("content", {}).get("markdown", "").strip()
        page = element.get("page", "unknown")
        element_id = element.get("id", "unknown")

        metadata = {
            "type": category,
            "page": page,
            "source": pdf_filename,
            "id": element_id,
            "paper_title": paper_title,
            "author": author,
            "summary": None,
            "key_words": key_words,
            "content": text_content
        }

        if category == "table":
            metadata.update({
                "png_file_path": element.get("png_filepath", None),
                "table_csv_path": element.get("table_csv_path", None)
            })

        documents.append(Document(page_content=markdown_content, metadata=metadata))

# ✅ 완료 로그
print(f"✅ 생성된 문서 개수: {len(documents)}")
if documents:
    print(f"🔹 첫 번째 문서 예시:\n{documents[0]}")

# PubMed 전문 upload

In [ ]:
import pandas as pd
from langchain_core.documents import Document
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# ✅ 엑셀 데이터 로드
excel_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\마취관련_키워드_pubmed_abstract_extraction_sample.xlsx"
df = pd.read_excel(excel_path)

# ✅ Document 생성 및 키워드 추출
for i, row in df.iterrows():
    abstract_text = str(row["article_abstract"]).strip()
    title = str(row["article_title"]).strip()
    year = row["year"]
    url = row["article_url"]

    metadata = {
        "type": "pubmed_abstract",
        "paper_title": title,
        "year": year,
    }

    doc = Document(page_content=abstract_text, metadata=metadata)
    documents.append(doc)

print(f"✅ 엑셀 기반 문서 {len(df)}개 생성 완료 (키워드 포함)")


In [ ]:
print(documents[0])

# Chunking 
- RecursiveChuncker
- SemanticChuncker - 채택

In [ ]:
from collections import defaultdict
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import SentenceTransformerEmbeddings

# ✅ LangChain 호환 SentenceTransformer 로드
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ SemanticChunker 설정
semantic_splitter = SemanticChunker(embeddings=embedding_model)

# ✅ 페이지별 텍스트 통합
page_texts = defaultdict(str)
page_metadata = {}

for doc in documents:
    doc_type = doc.metadata.get("type")
    page = doc.metadata.get("page", "pubmed")  # 엑셀 문서는 페이지 없음 → 임의 키

    key = (doc.metadata.get("source"), page)

    if doc_type in ["paragraph", "list", "table"] or doc_type == "pubmed_abstract":
        page_texts[key] += doc.page_content.strip() + " "
        page_metadata[key] = doc.metadata

# ✅ 의미 기반 청킹
semantic_docs = []

for key, full_text in page_texts.items():
    try:
        chunks = semantic_splitter.split_text(full_text)
        meta = page_metadata[key]
        for chunk in chunks:
            semantic_docs.append(Document(page_content=chunk.strip(), metadata=meta))
    except Exception as e:
        print(f"❌ 오류 (key={key}): {e}")

print(f"✅ 페이지 기반 청크 개수: {len(semantic_docs)}")
print(f"🔹 예시:\n{semantic_docs[0]}")


# Embeding & Vector DB 저장

In [ ]:
from pinecone import Pinecone, ServerlessSpec

# ✅ 환경변수에서 API 키 불러오기
api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("❌ PINECONE_API_KEY 환경변수가 설정되지 않았습니다.")

# ✅ Pinecone 클라이언트 생성
pc = Pinecone(api_key=api_key)

# ✅ 인덱스 이름 및 설정
index_name = "quickstart"
dimension = 1536  # 예: OpenAI embedding 모델 "text-embedding-ada-002" 사용 시

# ✅ 인덱스 생성 (이미 존재하는 경우 생략)
if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ 인덱스 '{index_name}' 생성 완료!")
else:
    print(f"✅ 인덱스 '{index_name}' 이미 존재합니다.")


In [ ]:
from openai import OpenAI
from uuid import uuid4
from pinecone import Pinecone
import os
from tqdm import tqdm

# ✅ API 키 불러오기
openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(api_key=openai_api_key)
index = pc.Index("quickstart") 

# ✅ 메타데이터 정리 함수
def clean_metadata(meta: dict) -> dict:
    cleaned = {}
    for k, v in meta.items():
        if v is None:
            continue
        if isinstance(v, (str, int, float, bool)):
            cleaned[k] = v
        elif isinstance(v, list) and all(isinstance(i, str) for i in v):
            cleaned[k] = v
        else:
            cleaned[k] = str(v)
    return cleaned

# ✅ 업로드 루프
batch_size = 50
vectors = []

for i, doc in enumerate(tqdm(semantic_docs)):
    try:
        # 💡 임베딩 생성
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=doc.page_content
        )
        embedding = response.data[0].embedding

        metadata = clean_metadata(doc.metadata)

        metadata["page_content"] = doc.page_content

        vectors.append({
            "id": str(uuid4()),
            "values": embedding,
            "metadata": metadata
        })

        # 📨 배치 업로드
        if len(vectors) == batch_size:
            index.upsert(vectors)
            vectors = []

    except Exception as e:
        print(f"❌ 에러 발생 (i={i}): {e}")

# 🔁 남은 벡터 업로드
if vectors:
    index.upsert(vectors)
    print("✅ 남은 벡터 업로드 완료")


print("✅ 임베딩 벡터 저장 완료!")

### 재시작할 때는 여기부터 실행하면 됨.

In [ ]:
import json
from pinecone import Pinecone

# ✅ Pinecone 연결
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("quickstart")

# ✅ 💾 저장된 벡터 불러오기
with open("cached_embeddings.json", "r") as f:
    vectors = json.load(f)

# ✅ Pinecone에 재업로드
index.upsert(vectors)
print("✅ 저장된 임베딩 재업로드 완료!")


# Retriever

In [ ]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone as NativePinecone
import os

# ✅ API 키
openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]

# ✅ 임베딩 모델
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

# ✅ Pinecone 클라이언트 & 인덱스
pc = NativePinecone(api_key=pinecone_api_key)
index = pc.Index("quickstart")

# ✅ LangChain VectorStore로 래핑
vectorstore = LangchainPinecone(index=index, embedding=embedding_model, text_key="page_content")

# ✅ Retriever 사용
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# ✅ 쿼리
query = "pediatric anesthesia guidelines"
docs = retriever.get_relevant_documents(query)

# ✅ 출력
if docs:
    print("📄 내용:", docs[0].page_content)
    print("📝 메타데이터:", docs[0].metadata)
else:
    print("❌ 문서를 찾을 수 없습니다.")



# Langgraph

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from typing import Annotated, List, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.documents import Document

class ChatbotState(TypedDict):
    question: Annotated[str, "Question"]
    documents: Annotated[List[Document], "Context"]
    chatbot: Annotated[str, "Answer"]
    messages: Annotated[List, add_messages]

In [ ]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone as NativePinecone
import os

openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]
langsmith_api_key = os.environ["LANGSMITH_API_KEY"]

embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
pc = NativePinecone(api_key=pinecone_api_key)
index = pc.Index("quickstart")

vectorstore = LangchainPinecone(index=index, embedding=embedding_model, text_key="page_content")
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

- 프롬프트 정의

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

# Ollama 기반 exaone 모델 설정
model = ChatOllama(model='llama3.2', temperature=0.5)

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant specialized in question-answering tasks based on medical research papers.
Use the following pieces of retrieved context to answer the question. If you don't know the answer, simply say that you don't know.
Answer in **Korean**.

# Direction:
1. Understand the intent of the question and provide the most accurate answer.
2. Identify and select the most relevant content from the retrieved context that directly relates to the question.
3. Construct a concise and logical answer by rearranging the selected information into coherent paragraphs.
4. If there is no relevant context for the question, state: "I can't find an answer to that question in the materials I have."
5. Present your answer in a table of key points where applicable.
6. Include all sources and their corresponding whole page numbers in your answer.
7. Write your answer entirely in **Korean**.
8. Be as detailed as possible in your answer.
9. Begin your answer with "This answer is based on content found in the document **📚" and end with "**📌 [document_name]" — here, [document_name] should be replaced with the document_name from the metadata.
10. Page numbers should be whole numbers.

#Context:
{context}

#Question:
{question}

#Answer:"""
)



In [ ]:
print(model.model)

- 노드 정의(문서 검색 & 답변 생성)

In [ ]:
from langchain_community.document_transformers import LongContextReorder
from langchain_core.messages import AIMessage
from langgraph.checkpoint.memory import MemorySaver

reorder = LongContextReorder()
memory = MemorySaver()

# ✅ 문서 검색 노드
def retrieve_document(state: ChatbotState):
    question = state["question"]
    docs = retriever.invoke(question)  # 너의 Pinecone 기반 retriever
    ordered = reorder.transform_documents(docs)
    return ChatbotState(documents=ordered)

# ✅ 답변 생성 노드
def llm_answer(state: ChatbotState):
    question = state["question"]
    docs = state["documents"]

    # context 정리: 요약 + 출처 포함
    formatted_contexts = []
    for doc in docs:
        source = doc.metadata.get("source", "Unknown source")
        page = doc.metadata.get("page", "Unknown page")
        content = doc.page_content.strip()
        formatted_contexts.append(f"{content}\n(Source: {source}, Page: {page})")
    
    context_str = "\n\n".join(formatted_contexts)

    prompt_text = prompt.format(context=context_str, question=question)
    response = model.invoke(prompt_text)

    if isinstance(response, AIMessage):
        content = response.content
    else:
        content = str(response)

    return ChatbotState(
        chatbot=content,
        messages=[("user", question), ("assistant", content)]
    )


- Graph compile

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(ChatbotState)

graph_builder.add_node("docs", retrieve_document)
graph_builder.add_node("llm_answer", llm_answer)

graph_builder.add_edge(START, "docs")
graph_builder.add_edge("docs", "llm_answer")
graph_builder.add_edge("llm_answer",END)

graph = graph_builder.compile(checkpointer=memory)


In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(graph)

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": "1"})
question = "What is the risk of VTE?"

for event in graph.stream(ChatbotState(question=question), config=config):
    print(event)

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": "1"})
question = "What factors contribute to the scarcity of data on oxygen management in pediatric anesthesia?"

for event in graph.stream(ChatbotState(question=question), config=config):
    print(event)